In [1]:
root = 'experiment2/'

In [1]:
# coding:utf-8

# 导入模块
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
import pickle
from bilstm import BiLSTM

# 日志等级
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# Parameters
# ==================================================

# Data loading params
# 数据集划分：4(训练集）:1（验证集）:5（测试集）
tf.flags.DEFINE_float("valid_sample_percentage", .1, "Percentage of the dataset to use for validation")
tf.flags.DEFINE_float("test_sample_percentage", .5, "Percentage of the dataset to use for test")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_float("dropout_keep_prob", 0.4, "Dropout keep probability (default: 0.5)")                        # 调整
tf.flags.DEFINE_float("learning_rate", 0.001, "Learning rate")                                                    # 调整

# Training parameters
tf.flags.DEFINE_integer("batch_size", 128, "Batch Size (default: 64)")                                            # 调整
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")                            # 调整
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 7800, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 1, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

# FLAGS
FLAGS = tf.flags.FLAGS

# 最高准确率
max_train_acc = 0
max_valid_acc = 0


# 加载预训练结果
# load_obj
# 函数功能：加载对象
def load_obj(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)


# 存储<token,序号>的字典
dictionary = load_obj("./dataset/word2vec/dictionary.pkl")


# preprocess
# 函数功能：加载数据集
def preprocess():
    # Load data
    print("Loading data...")
    x, y = data_helpers.load_data_and_labels(FLAGS.positive_data_file, FLAGS.negative_data_file)

    # Randomly shuffle data
    np.random.seed(25)  # 种子决定随机序列（种子固定，shuffle出来的就是固定的）
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # Split train/validation/test set (after shuffle)
    valid_sample_index = -1 * int((FLAGS.valid_sample_percentage + FLAGS.test_sample_percentage) * float(len(y)))
    test_sample_index = -1 * int(FLAGS.test_sample_percentage * float(len(y)))
    x_train, x_valid, x_test = x_shuffled[:valid_sample_index], x_shuffled[valid_sample_index:test_sample_index], x_shuffled[test_sample_index:]
    y_train, y_valid, y_test = y_shuffled[:valid_sample_index], y_shuffled[valid_sample_index:test_sample_index], y_shuffled[test_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print("Train/Validation/Test split: {:d}/{:d}/{:d}".format(len(y_train), len(y_valid), len(y_test)))
    return x_train, y_train, x_valid, y_valid, x_test, y_test


# train
# 函数功能：训练
def train(x_train, y_train, x_valid, y_valid, x_test, y_test):
    # Training
    # ==================================================
    print('start training...')
    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=FLAGS.allow_soft_placement,
          log_device_placement=FLAGS.log_device_placement)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            lstm = BiLSTM(
                sequence_length=x_train.shape[1],  # 指令数
                num_classes=y_train.shape[1])

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)
            grads_and_vars = optimizer.compute_gradients(lstm.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", lstm.loss)
            acc_summary = tf.summary.scalar("accuracy", lstm.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

            # Initialize all variables
            sess.run(tf.local_variables_initializer())  # 计算FP、FN、TP、TN必须要初始化局部变量
            sess.run(tf.global_variables_initializer())

            # 训练
            def train_step(x_batch, y_batch):
                global max_train_acc
                """
                A single training step
                """
                feed_dict = {
                  lstm.actual_batch_size: len(x_batch),
                  lstm.input_x: x_batch,
                  lstm.input_y: y_batch,
                  lstm.dropout_keep_prob: FLAGS.dropout_keep_prob
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, lstm.loss, lstm.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                if accuracy > max_train_acc:
                    max_train_acc = accuracy
                if step % 20 == 0:
                    print("{}: step {}, loss {:g}, train_acc {:g}, max_train_acc {:g}".format(time_str, step, loss, accuracy, max_train_acc))
                train_summary_writer.add_summary(summaries, step)

            # 验证
            def valid_step(x_batch, y_batch):
                global max_valid_acc
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  lstm.actual_batch_size: len(x_batch),
                  lstm.input_x: x_batch,
                  lstm.input_y: y_batch,
                  lstm.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, lstm.loss, lstm.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                if accuracy > max_valid_acc:
                    max_valid_acc = accuracy
                print("{}: step {}, loss {:g}, valid_acc {:g}, max_valid_acc {:g}".format(time_str, step, loss, accuracy, max_valid_acc))

            # 测试
            def test_step(x_batch, y_batch):
                feed_dict = {
                    lstm.actual_batch_size: len(x_batch),
                    lstm.input_x: x_batch,
                    lstm.input_y: y_batch,
                    lstm.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, lstm.loss, lstm.accuracy],
                    feed_dict)
                TP, FP, TN, FN = sess.run([lstm.TP, lstm.FP, lstm.TN, lstm.FN], feed_dict=feed_dict)
                # 取出Value
                TP = TP[1]
                FP = FP[1]
                TN = TN[1]
                FN = FN[1]
                # 计算Matrix
                mP = TP / (TP + FP)
                mR = TP / (TP + FN)
                mF1_score = 2 * mP * mR / (mP + mR)
                print("loss {:g}, test_acc {:g}, test_recall {:g}, test_precision {:g}, test_F1 {:g}".
                      format(loss, accuracy, mR, mP, mF1_score))

            # Generate batches
            batches = data_helpers.batch_iter(
                list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)

            print('training... ')
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % FLAGS.evaluate_every == 0:
                    print("\nEvaluation:")
                    valid_step(x_valid, y_valid)
                    print("")
                if current_step % FLAGS.checkpoint_every == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))
            # Test
            print("\nTest:")
            test_step(x_test, y_test)
            print("")


def main(argv=None):
    x_train, y_train, x_valid, y_valid, x_test, y_test = preprocess()
    train(x_train, y_train, x_valid, y_valid, x_test, y_test)


# if __name__ == '__main__':
#     tf.app.run()


In [2]:
tf.app.run()

Loading data...
数据集文件./dataset/CWE121_bad_dataset_noUJA.txt加载完毕..
分解结构加载完毕..
正样本加载完毕..
数据集文件./dataset/CWE121_good_dataset_noUJA.txt加载完毕..
分解结构加载完毕..
负样本加载完毕..
Train/Validation/Test split: 2807/701/3508
start training...
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

INFO:tensorflow:Summary name bidirectional_rnn/fw/basic_lstm_cell/kernel:0/grad/hist is illegal; using bidirectional_rnn/fw/basic_lstm_cell/kernel_0/grad/hist instead.
INFO:tensorflow:Summary name bidirectional_rnn/fw/basic_lstm_cell/kernel:0/grad/sparsity is illegal; using bidirectional_rnn/fw/basic_lstm_cell/kernel_0/grad/sparsity instead.
INFO:tensorfl

SystemExit: 

/home/bfs/anaconda3/envs/tensorflow/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
